### Days: wps_climdex_days

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) days functions

- [climdex.su](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=25) Computes the SU (summer days) climdexindex: the annual count of days where daily maximum temperature exceeds 25 degreesCelsius.
- [climdex.id](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=11) Computes the ID (icing days) climdexindex: the annual count of days where daily maximum temperature was below 0 degrees Celsius.
- [climdex.fd](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=8) Computes the FD frost days) climdexindex: the annual count of days where daily minimum temperature was below 0 degrees Celsius.
- [climdex.tr](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=32) Computees the TR (tropical nights) climdexindex: the annual count of days where daily minimum temperature stays above 20 degrees Celsius.

In [ ]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url
from quail.utils import test_rda_output

In [ ]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [ ]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

In [ ]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [ ]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_days?

#### Run wps_climdex_days process for climdex.su (summer days)

In [ ]:
with NamedTemporaryFile(suffix=".rda", prefix="summer_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",
            days_type="summer days",
            vector_name="summer_days",
            output_file=output_file.name,
        )
su_url = output.get()[0]

#### Run wps_climdex_days process for climdex.id (icing days)

In [ ]:
with NamedTemporaryFile(suffix=".rda", prefix="icing_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",        
            days_type="icing days",
            vector_name="icing_days",
            output_file=output_file.name,
        )
id_url = output.get()[0]

#### Run wps_climdex_days process for climdex.fd (frost days)

In [ ]:
with NamedTemporaryFile(suffix=".rda", prefix="frost_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",        
            days_type="frost days",
            vector_name="frost_days",
            output_file=output_file.name,
        )
fd_url = output.get()[0]

#### Run wps_climdex_days process for climdex.tr (tropical nights)

In [ ]:
with NamedTemporaryFile(suffix=".rda", prefix="frost_days_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_days(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",        
            days_type="tropical nights",
            vector_name="tropical_nights",
            output_file=output_file.name,
        )
tr_url = output.get()[0]

Access the output with **rda_to_vector()** or **auto_construct_outputs()** from **wps_tools.output_handling**

In [ ]:
from wps_tools.output_handling import rda_to_vector, auto_construct_outputs

# use print() to see whole vector
su = rda_to_vector(su_url, "summer_days")
print(f"SUMMER DAYS\n{su}")
id_ = rda_to_vector(id_url, "icing_days")
print(f"ICING DAYS\n{id_}")
fd = rda_to_vector(fd_url, "frost_days")
print(f"FROST DAYS\n{fd}")

In [ ]:
auto_construct_outputs([su_url, id_url, fd_url])

#### Test output against expected output

In [ ]:
test_rda_output(
        su_url, "summer_days", "expected_days_data.rda", "expected_summer_days"
    )
test_rda_output(
        id_url, "icing_days", "expected_days_data.rda", "expected_icing_days"
    )
test_rda_output(
        fd_url, "frost_days", "expected_days_data.rda", "expected_frost_days"
    )